# Introduction to Community Modeling

Author: Daniel Machado, NTNU

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).

- You will use the [ReFramed](https://github.com/cdanielmachado/reframed) python library for metabolic modeling. 

In [1]:
from reframed.solvers import solvers

solvers

{'gurobi': reframed.solvers.gurobi_solver.GurobiSolver,
 'optlang': reframed.solvers.optlang_solver.OptLangSolver}

## Step 1: Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

![synthetic community](../files/synthetic_community.png)

As usual, we start by loading the model for the wild-type:

In [1]:
from reframed import load_cbmodel
from cobra.io import read_sbml_model

In [2]:
bt = load_cbmodel('../models/non-ec/ncbi_refseq/vpi5482.xml')

In [3]:
bu = load_cbmodel('../models/non-ec/ncbi_refseq/atcc8492.xml')

In [4]:
ec = load_cbmodel('../models/non-ec/ncbi_refseq/ed1a.xml')

In [5]:
cc = load_cbmodel('../models/non-ec/ncbi_refseq/atcc27758.xml')

#fn = load_cbmodel('../models/non-ec/Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586.xml')

ri = load_cbmodel('../models/non-ec/ncbi_refseq/l182.xml')

sp = load_cbmodel('../models/non-ec/ncbi_refseq/atcc15912.xml')

#ss = load_cbmodel('../models/non-ec/Streptococcus_salivarius_DSM_20560.xml')

In [6]:
from reframed import Community
community = Community('sample1', [bt,bu,ec,cc,ri,sp])

This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [7]:
from reframed import Environment

M9 = Environment.from_model(bt)

print(f"Environment compounds: {', '.join(M9.get_compounds())}")

Environment compounds: 12ppd__S, 14glucan, 2ameph, 2m35mdntha, 2pglyc, 35dnta, 4abut, 5drib, 5mcsn, LalaDgluMdap, LalaLglu, Larab, R3hdec4e, R_3hdcaa, R_3hphxa, R_3hppta, R_3hpt, abg4, acald, acmana, actn__R, ad, ade, adn, ala__D, ala__L, alaala, alltn, anhgm, arab__L, arg__L, asn__L, aso3, aso4, asp__L, btn, bz, ca2, cellb, cgly, chol, chols, cit, cl, co2, coa, cobalt2, crn, csn, cu2, cys__L, cytd, d23hb, dad_2, dca, dcyt, dgsn, din, drib, duri, eths, fald, fe2, fe3, fe3dcit, fe3dhbzs3, fe3pyovd_kt, fol, fuc__L, g3pg, gal_bD, gal, galt, galur, gcald, glc__D, glcur, gln__L, glu__L, glucan4, glucan6, glutar, gly, glyb, glyc3p, gthox, gthrd, gua, h2, h2o2, h2o, h2s, h, ham, his__L, hxa, ile__L, indole, inost, isobuta, istfrnA, istfrnB, k, lac__D, lcts, leu__L, lipoate, lmn2, lys__L, lyx__L, m_xyl, mal__L, malttr, manttr, melib, meoh, met__D, met__L, metox__R, metox, mg2, mn2, n2o, ncam, nh4, no2, no, o2, ocdcea, octa, p_xyl, phe__L, phehpa, pheme, pheocta, pi, pime, ppap, ppi, pro__L, pt

In [8]:
M9

R_sink_4crsol_c	0.0	inf
R_sink_4hba_c	0.0	inf
R_sink_amob_c	0.0	inf
R_sink_lipopb_c	0.0	inf
R_EX_12ppd__S_e	-inf	inf
R_EX_14glucan_e	-inf	inf
R_EX_2ameph_e	-inf	inf
R_EX_2m35mdntha_e	-inf	inf
R_EX_2pglyc_e	-inf	inf
R_EX_35dnta_e	-inf	inf
R_EX_4abut_e	-inf	inf
R_EX_5drib_e	-inf	inf
R_EX_5mcsn_e	-inf	inf
R_EX_LalaDgluMdap_e	-inf	inf
R_EX_LalaLglu_e	-inf	inf
R_EX_Larab_e	-inf	inf
R_EX_R3hdec4e_e	-inf	inf
R_EX_R_3hdcaa_e	-inf	inf
R_EX_R_3hphxa_e	-inf	inf
R_EX_R_3hppta_e	-inf	inf
R_EX_R_3hpt_e	-inf	inf
R_EX_abg4_e	-inf	inf
R_EX_acald_e	-inf	inf
R_EX_acmana_e	-inf	inf
R_EX_actn__R_e	-inf	inf
R_EX_ad_e	-inf	inf
R_EX_ade_e	-inf	inf
R_EX_adn_e	-inf	inf
R_EX_ala__D_e	-inf	inf
R_EX_ala__L_e	-inf	inf
R_EX_alaala_e	-inf	inf
R_EX_alltn_e	-inf	inf
R_EX_anhgm_e	-inf	inf
R_EX_arab__L_e	-inf	inf
R_EX_arg__L_e	-inf	inf
R_EX_asn__L_e	-inf	inf
R_EX_aso3_e	-inf	inf
R_EX_aso4_e	-inf	inf
R_EX_asp__L_e	-inf	inf
R_EX_btn_e	-inf	inf
R_EX_bz_e	-inf	inf
R_EX_ca2_e	-inf	inf
R_EX_cellb_e	-inf	inf
R_EX_cgly_e	-inf	in

## Step 2: Simulation using (conventional) FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [9]:
from reframed import FBA

super_organism = community.merged_model
solution = FBA(super_organism, constraints=M9)

print(solution)
solution.show_values(pattern='R_EX')

Set parameter Username
Objective: None
Status: Infeasible or Unbounded



/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_sink_4crsol_c' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_sink_4hba_c' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_sink_amob_c' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_sink_lipopb_c' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")


We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the (non-zero) fluxes for each organism:

In [10]:
solution.show_values(pattern='_ec', sort=True)

In [11]:
solution.show_values(pattern='_nh4_ko', sort=True)

In [58]:
solution.show_values(pattern='_bt', sort=True)

## Step 3: Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [59]:
from reframed import SteadyCom
solution = SteadyCom(community, constraints=M9)

AttributeError: 'gurobipy.Model' object has no attribute 'linear_constraints'

Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [37]:
print(solution)

Objective: -0.0
Status: Optimal



The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [ ]:
solution.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [38]:
from reframed import fluxes2escher
fluxes2escher(solution.internal['glc_ko'])

AttributeError: 'Solution' object has no attribute 'internal'

In [ ]:
fluxes2escher(solution.internal['nh4_ko'])

**Exercise:** Look more closely at the compounds that are exchanged between the two mutants and also at their relative abundance. Is this what you expected? Do you think there could be different solutions?

## Step 4: Explore alternative solutions

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

![variability](../files/steadycom_variability.png)

> Would you expect a synthetic community to grow at its maximum growth rate?

**ReFramed** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [ ]:
from reframed import SteadyComVA
variability = SteadyComVA(community, obj_frac=0.9, constraints=M9)

print('Strain\tMin\tMax')
for strain, (lower, upper) in variability.items():
    print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

In [ ]:
# Feel free to play around with these examples.
# Type your own code here...